In [4]:
%load_ext autoreload
%autoreload 2

# Feature Pipeline: Computing features

In [5]:
import os
import sys

repo_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_path)

import torch
import warnings
import polars as pl

from pprint import pprint
from loguru import logger
from sentence_transformers import SentenceTransformer

# Config
from recsys.config import settings

# Feature Store
# from recsys import gcp_integrations
# from recsys.gcp_integrations import feature_store

# Features
from recsys.features.articles import (
    compute_features_articles,
    generate_embeddings_for_dataframe,
)
from recsys.features.customers import DatasetSampler, compute_features_customers
from recsys.features.interaction import generate_interaction_data
from recsys.features.transactions import compute_features_transactions

# Raw Data
from recsys.raw_data_sources import h_and_m_raw_data

warnings.filterwarnings("ignore")

/opt/homebrew/Caskroom/miniconda/base/envs/mlops/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 🗄️ Articles data

The **article_id** and **product_code** serve different purposes in the context of H&M's product database:

- **Article ID**: This is a unique identifier assigned to each individual article within the database. It is typically used for internal tracking and management purposes. Each distinct item or variant of a product (e.g., different sizes or colors) would have its own unique article_id.

- **Product Code**: This is also a unique identifier, but it is associated with a specific product or style rather than individual articles. It represents a broader category or type of product within H&M's inventory. Multiple articles may share the same product code if they belong to the same product line or style.

While both are unique identifiers, the article_id is specific to individual items, whereas the product_code represents a broader category or style of product.

Here is an example:

**Product: Basic T-Shirt**

- **Product Code:** TS001

- **Article IDs:**
    - Article ID: 1001 (Size: Small, Color: White)
    - Article ID: 1002 (Size: Medium, Color: White)
    - Article ID: 1003 (Size: Large, Color: White)
    - Article ID: 1004 (Size: Small, Color: Black)
    - Article ID: 1005 (Size: Medium, Color: Black)

In this example, "TS001" is the product code for the basic t-shirt style. Each variant of this t-shirt (e.g., different sizes and colors) has its own unique article_id.



In [6]:
articles_df = h_and_m_raw_data.extract_articles_df()
articles_df.shape

(105542, 25)

In [7]:
articles_df.head(3)

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
i64,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,str
108775015,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775044,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775051,108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"


## Articles feature engineering

In [8]:
articles_df = compute_features_articles(articles_df)
articles_df.shape

(105542, 27)

In [9]:
articles_df.head(3)

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,prod_name_length,article_description,image_url
str,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,u32,str,str
"""108775015""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""gs://gonzo-recsys-data/h-and-m…"
"""108775044""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""gs://gonzo-recsys-data/h-and-m…"
"""108775051""",108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",13,"""Strap top (1) - Vest top in Ga…","""gs://gonzo-recsys-data/h-and-m…"


## Create embeddings from the articles description

In [10]:
for i, desc in enumerate(articles_df["article_description"].head(n=3)):
    logger.info(f"Item {i + 1}:\n{desc}")

2025-02-05 23:04:59.591 | INFO     | __main__:<module>:2 - Item 1:
Strap top - Vest top in Garment Upper body
 Apperance: Solid
 Color: Dark Black 9
 Category: Ladieswear Womens Everyday Basics Jersey Basic
 Details: Jersey top with narrow shoulder straps.
2025-02-05 23:04:59.591 | INFO     | __main__:<module>:2 - Item 2:
Strap top - Vest top in Garment Upper body
 Apperance: Solid
 Color: Light White 10
 Category: Ladieswear Womens Everyday Basics Jersey Basic
 Details: Jersey top with narrow shoulder straps.
2025-02-05 23:04:59.592 | INFO     | __main__:<module>:2 - Item 3:
Strap top (1) - Vest top in Garment Upper body
 Apperance: Stripe
 Color: Dusty Light White 11
 Category: Ladieswear Womens Everyday Basics Jersey Basic
 Details: Jersey top with narrow shoulder straps.


In [11]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

logger.info(
    f"Loading ${settings.FEATURES_EMBEDDING_MODEL_ID} embedding model to {device=}"
)

# Load embedding model from SentenceTransformers model registry
model = SentenceTransformer(settings.FEATURES_EMBEDDING_MODEL_ID, device=device)

2025-02-05 23:04:59.627 | INFO     | __main__:<module>:9 - Loading $all-MiniLM-L6-v2 embedding model to device='mps'


In [12]:
articles_df = generate_embeddings_for_dataframe(
    articles_df, "article_description", model, batch_size=128
)

Generating embeddings...: 100%|██████████| 105542/105542 [05:33<00:00, 316.48it/s]


For each article description, we have a numerical vector which we can feed to a model, opposite to a string containing the description of an object.

In [13]:
articles_df[["article_description", "embeddings"]].head(3)

article_description,embeddings
str,list[f64]
"""Strap top - Vest top in Garmen…","[-0.023064, 0.069894, … 0.012397]"
"""Strap top - Vest top in Garmen…","[-0.011304, 0.075073, … 0.016485]"
"""Strap top (1) - Vest top in Ga…","[-0.033244, 0.081816, … 0.020111]"


In [14]:
articles_df["image_url"][0]

'gs://gonzo-recsys-data/h-and-m/images/010/0108775015.jpg'

In [15]:
from IPython.display import HTML, display

image_urls = articles_df["image_url"].tail(12).to_list()
grid_html = '<div style="display: grid; grid-template-columns: repeat(6, 1fr); gap: 10pxl max-width: 900px;">'

for url in image_urls:
    grid_html += f'<img src="{url}" style="width: 100%; height: auto;">'

grid_html += "</div>"

display(HTML(grid_html))

# Customers Data

In [16]:
customers_df = h_and_m_raw_data.extract_customers_df()
customers_df.shape

(1371980, 7)

In [17]:
customers_df.head(3)

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
str,f64,f64,str,str,i64,str
"""00000dbacae5abe5e23885899a1fa4…",null,null,"""ACTIVE""","""NONE""",49,"""52043ee2162cf5aa7ee79974281641…"
"""0000423b00ade91418cceaf3b26c6a…",null,null,"""ACTIVE""","""NONE""",25,"""2973abc54daa8a5f8ccfe9362140c6…"
"""000058a12d5b43e67d225668fa1f8d…",null,null,"""ACTIVE""","""NONE""",24,"""64f17e6a330a85798e4998f62d0930…"


In [18]:
customers_df.null_count()

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
u32,u32,u32,u32,u32,u32,u32
0,895050,907576,6062,16009,15861,0


## Customers feature engineering

In [19]:
customers_df.head(3)

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
str,f64,f64,str,str,i64,str
"""00000dbacae5abe5e23885899a1fa4…",null,null,"""ACTIVE""","""NONE""",49,"""52043ee2162cf5aa7ee79974281641…"
"""0000423b00ade91418cceaf3b26c6a…",null,null,"""ACTIVE""","""NONE""",25,"""2973abc54daa8a5f8ccfe9362140c6…"
"""000058a12d5b43e67d225668fa1f8d…",null,null,"""ACTIVE""","""NONE""",24,"""64f17e6a330a85798e4998f62d0930…"


In [20]:
customers_df = compute_features_customers(customers_df, drop_null_age=True)
customers_df.shape

(1356119, 5)

In [21]:
customers_df.head(3)

customer_id,club_member_status,age,postal_code,age_group
str,str,f64,str,str
"""00000dbacae5abe5e23885899a1fa4…","""ACTIVE""",49.0,"""52043ee2162cf5aa7ee79974281641…","""46-55"""
"""0000423b00ade91418cceaf3b26c6a…","""ACTIVE""",25.0,"""2973abc54daa8a5f8ccfe9362140c6…","""19-25"""
"""000058a12d5b43e67d225668fa1f8d…","""ACTIVE""",24.0,"""64f17e6a330a85798e4998f62d0930…","""19-25"""


# Transactions Data

In [22]:
transactions_df = h_and_m_raw_data.extract_transactions_df()
transactions_df.shape

(31788324, 5)

In [23]:
transactions_df.head(3)

t_dat,customer_id,article_id,price,sales_channel_id
date,str,i64,f64,i64
2018-09-20,"""000058a12d5b43e67d225668fa1f8d…",663713001,0.050831,2
2018-09-20,"""000058a12d5b43e67d225668fa1f8d…",541518023,0.030492,2
2018-09-20,"""00007d2de826758b65a93dd24ce629…",505221004,0.015237,2


## Transactions feature engineering

In [24]:
transactions_df = compute_features_transactions(transactions_df)
transactions_df.shape

(31788324, 9)

The time of the year a purchase was made should be a strong predictor, as seasonality plays a big factor in fashion purchases. Here, you will use the month of the purchase as a feature. Since this is a cyclical feature (January is as close to December as it is to February), you'll map each month to the unit circle using sine and cosine.

Thus, the features of the transactions DataFrame look as follows:

In [25]:
transactions_df.head(3)

t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,day_of_week
i64,str,str,f64,i64,i32,i8,i8,i8
0,"""000058a12d5b43e67d225668fa1f8d…","""663713001""",0.050831,2,2018,9,20,4
0,"""000058a12d5b43e67d225668fa1f8d…","""541518023""",0.030492,2,2018,9,20,4
0,"""00007d2de826758b65a93dd24ce629…","""505221004""",0.015237,2,2018,9,20,4


We don't want to work with ~30 million transactions in these series, as everything will take too much time to run. Thus, we create a subset of the original dataset by randomly sampling from the customers' datasets and taking only their transactions.

In [26]:
sampler = DatasetSampler(size=settings.CUSTOMER_DATA_SIZE)
dataset_subset = sampler.sample(
    customers_df=customers_df, transactions_df=transactions_df
)
customers_df = dataset_subset["customers"]
transactions_df = dataset_subset["transactions"]

2025-02-05 23:18:07.715 | INFO     | recsys.features.customers:sample:29 - Sampling 1000 customers.
2025-02-05 23:18:07.753 | INFO     | recsys.features.customers:sample:32 - Number of transactions for all the customers: 31788324
2025-02-05 23:18:08.384 | INFO     | recsys.features.customers:sample:38 - Number of transactions for the 1000 sampled customers: 23799


In [27]:
transactions_df.shape

(23799, 9)

Some of the remaining customers:

In [28]:
for customer_id in transactions_df["customer_id"].unique().head(10):
    logger.info(f"Logging customer ID: {customer_id}")

2025-02-05 23:18:08.489 | INFO     | __main__:<module>:2 - Logging customer ID: c105fdd5ec36bef99e989ac0a9e725f5fe68a7765d3a6fc426e947f0b73dbb9d
2025-02-05 23:18:08.489 | INFO     | __main__:<module>:2 - Logging customer ID: 5790bdcb9c0d4ea350622bb2cff1da5d37786c2ed2388000e58738500eb37dfc
2025-02-05 23:18:08.489 | INFO     | __main__:<module>:2 - Logging customer ID: 15b2237d5ba7770d7cbcdeac6628ad4d2030ca35e46667bb63914dcbc3f9aa27
2025-02-05 23:18:08.489 | INFO     | __main__:<module>:2 - Logging customer ID: b1fdc1db25e8539c8a8b1a390dcd03c86e53bb4407ef6e237cbb7b8dcb853f1f
2025-02-05 23:18:08.490 | INFO     | __main__:<module>:2 - Logging customer ID: 889d2c1d49250464b1c4a838864f53950b2bff8b2b8feeca80e7e7b0b38cd55f
2025-02-05 23:18:08.490 | INFO     | __main__:<module>:2 - Logging customer ID: 871f9e45eed0daca0e3e3a3d90f7153763b2e41737ad0568bb0633fd71d1ec92
2025-02-05 23:18:08.490 | INFO     | __main__:<module>:2 - Logging customer ID: 70ea0561e99d2e66da63cba206615b0341d160dc8e99600b74

# 🤳🏻 Interaction data

To train our models, we need more than just the transactions DataFrame. We need positive samples that signal whether a customer clicked or bought an item, but we also need negative samples that signal no interactions between a customer and an item.

In [29]:
interaction_df = generate_interaction_data(transactions_df)
interaction_df.shape

Processing customer chunks:   0%|          | 0/1 [00:03<?, ?it/s]


(134898, 5)

In [33]:
interaction_df.head()

t_dat,customer_id,article_id,interaction_score,prev_article_id
i64,str,str,i64,str
-339120000,"""00b203a32faa3d007dba198ef27c15…","""689096003""",0,"""START"""
-337320000,"""00b203a32faa3d007dba198ef27c15…","""623866002""",0,"""689096003"""
-335160000,"""00b203a32faa3d007dba198ef27c15…","""623866002""",0,"""623866002"""
-309600000,"""00b203a32faa3d007dba198ef27c15…","""568931002""",0,"""623866002"""
-309240000,"""00b203a32faa3d007dba198ef27c15…","""568931002""",0,"""568931002"""


Score distributions:

Here is what each score means:
- `0` : No interaction between a customer and an item
- `1` : A customer clicked an item
- `2` : A customer bought an item

In [32]:
interaction_df.group_by("interaction_score").agg(
    pl.count("interaction_score").alias("total_interactions")
)

interaction_score,total_interactions
i64,u32
0,73054
1,38045
2,23799


# Feature Groups

# Compute Ranking Dataset